# Задание 2. Бинарная классификация.
В этом задании вам предстоит пройти путь от анализа и обработки данных до построения модели машинного обучения для решения задачи бинарной классификации.

**Датасет**: космический корабль «Титаник» с почти 13 000 пассажирами на борту отправился в свой первый рейс, чтобы доставить эмигрантов из нашей Солнечной системы к трем новым обитаемым экзопланетам, вращающимся вокруг ближайших звезд. К сожалению, по пути судно натолкнулось на пространственно-временную аномалию, вследствие чего почти половина пассажиров была перенесена в другое измерение! Чтобы помочь спасателям, необходимо на основе имеющихся сведений научиться предсказывать, кто из пассажиров был перенесен, а кто остался на корабле. [Ссылка](https://www.kaggle.com/competitions/spaceship-titanic) на страницу соревнования на Kaggle.

**Описание признаков:**

- `PassengerId`: уникальный ID для каждого пассажира. Каждый идентификатор имеет формат gggg_pp, где gggg указывает группу, с которой путешествует пассажир, а pp — его номер в группе. Люди в группе часто являются членами семьи, но не всегда.
- `HomePlanet`: планета, с которой пассажир отправился в путешествие
- `CryoSleep`: указывает, был ли пассажир быть переведен в режим анабиоза на время рейса.
- `Cabin`: номер кабины.
- `Destination`: планета назначения.
- `Age`: возраст.
- `VIP`: оплатил ли пассажир специальное VIP-обслуживание во время рейса.
- `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck`: сумма, которую пассажир заплатил за различные удобства на борту.
- `Name`: Имя и фамилия.
- `Transported`: был ли пассажир перенесен в другое измерение. Это целевая переменная, которую необходимо предсказать.

- Импортируйте необходимые библиотеки. Загрузите датасет и отобразите первые несколько объектов. Разбейте для удобства исходный датафрейм на матрицу объектов-признаков $X$ и целевую переменную $y$

In [ ]:
# YOUR CODE HERE

- Для задания нам не понадобятся признаки `Cabin` и `Name`, отбросьте их. Отобразите, сколько признаков у нас осталось и сколько всего объектов в датасете.

In [ ]:
# YOUR CODE HERE

### Визуализация данных

Можете пользоваться как matplotlib, так и Seaborn

- Нарисуйте столбчатую диаграмму для признака `HomePlanet`. С какой планеты отправилось больше всего пассажиров?

In [ ]:
# YOUR CODE HERE

- Отобразите распределение таргета целевой переменной. Что можно о нем сказать? Оправдано ли использование метрики Accuracy в данной задаче?

In [ ]:
# YOUR CODE HERE

- Постройте распределения признаков `Age`, `RoomService` и `ShoppingMall` и найдите для каждого из них среднее, медиану, среднеквадратическое отклонение и 0.9-квантиль.

In [ ]:
# YOUR CODE HERE

### Подготовка данных

- Для проверки работы модели будем использовать **валидацию на отложенной выборке**: часть данных будет использоваться для тренировки алгоритма, другая (меньшая) часть для валидации.

    Воспользуйтесь функцией `train_test_split` из библиотеки scikit-learn для разделения исходных $X$ и $y$ на тренировочные и валидационные данные. Размер валидационной выборки установите равным 20 процентам от общего размера выборки.

In [ ]:
# YOUR CODE HERE

- Некоторые функции преобразования зависят от состава выборки. Как мы увидим далее, кодирование категориальных данных может зависеть от частоты появления той или иной категории в датасете, параметры StandartScaler также зависят от тренировочной выборки и т.д. Правильным подходом является "настройка" всех функций преобразований на тренировочном датасете, а затем применение их на валидационном датасете (подобно тому, как мы обучаем модель на трейне, а на валидационных данных только проверяем). Если применить все преобразования перед разделением датасета на train и validation, то мы столкнемся с проблемой **утечки данных (data leakage)**. В таком случае оценка нашей модели будет не совсем честной и искаженной (пусть даже если и немного). 

    В заданиях ниже необходимо будет преобразовывать как $X_{train}$, так и $X_{val}$
  
    Определите, какие из категориальных признаков в данных являются бинарными (принимают два значения). Преобразуйте их так, чтобы они принимали значения $0$ или $1$. Если в колонке имеются пропуски, замените их наиболее часто встречающимся значением (модой) в тренировочном датасете.

In [ ]:
# YOUR CODE HERE

- На лекции мы рассматривали один из самых распространенных и простых методов кодирования категориальных данных - **one-hot кодирование**. Мы использовали его, когда признак мог принимать несколько (больше двух) значений. Одним из минусов этого алгоритма является то, что мы добавляем новые колонки в наш датасет, тем самым расширяя признаковое пространство при неизменном количестве исходных объектов.

    Рассмотрим другой метод кодирования категориальных данных - **Frequency Encoding (частотное кодирование**). В этом подходе каждая категория (рассматриваем некоторый признак) заменяется на число, равное количеству объектов с этим значением категории делённое на количество всех объектов. Иными словами мы пытаемся найти, насколько часто соответствующая категория встречается в наших данных. Пример работы алгоритма:

    <center><img src="freq_enc_example.png" alt="drawing" width="200"/></center>

    Данный метод достаточно прост и эффективен в использовании, он также не создает множество новых признаков в отличие от one-hot кодировани. Однако есть и минус: если две или несколько категорий встречаются в данных примерно одинаковое количество раз, то и закодированные величины будут очень близки друг к другу. Это может привести к тому, что для модели будет тяжелее выявить отличия между категориями.

    **Реализуйте функцию**, которая будет принимать на вход тренировочный и валидационный датафреймы и список колонок для преобразования. Обратите внимание, что **в колонках могут быть пропущенные значения** (NaN), обработайте их так, будто это еще одна дополнительная категория (например, если бы в примере выше помимо цветов была еще категория 'NaN'). Преобразуйте с помощью этой функции колонки, в которых больше двух категорий. Использовать готовые реализации алгоритма нельзя.

In [ ]:
def freq_enc(X_train, X_val, col_enc):
    for col in col_enc:
       pass
       # YOUR CODE HERE

- На данном этапе у вас не должно остаться пропусков в категориальных признаках. Заполните пропуски в оставшихся колонках медианным значением соответствующего признака. Удостоверьтесь, что у вас больше нет пропусков ни в каком признаке.

In [ ]:
# YOUR CODE HERE
print(X_train.isna().sum())
print(X_val.isna().sum())

- Воспользуйтесь функцией `StandardScaler`, чтобы отмасштабировать данные по формуле $z = (x - \mu)/\sigma$, где $\mu$ - среднее значение тренировочных объектов, $\sigma$ - среднеквадратическое отклонение. Не забывайте, что настройка Scaler'а также производится на тренировочном датасете.

In [ ]:
# YOUR CODE HERE

### Обучение и выбор модели

- Данные подготовлены, теперь пришло время переходить к построению модели. Оцените качество логистической регрессии с $L2$-регуляризацией (sklearn.linear_model.LogisticRegression) и метода опорных векторов с ядром `rbf` (sklearn.svm.SVC). Задайте некоторый диапазон для гиперпараметров и найдите, при каких из них метрика качества (будем использовать **Accuracy**) наибольшая. Какой алгоритм показал наилучшие результаты?
  
  Отобразите лучшее значение метрики и при каких параметрах оно достигается. Добейтесь значения Accuracy не меньше 0.77 хотя бы с одним из алгоритмов.

In [ ]:
# Логистическая регрессия

# Коэффициент регуляризации
C_regul = []

for regul in C_regul:
    pass
    # YOUR CODE HERE

In [ ]:
# Метод опорных векторов
import itertools

# сетка со значенями параметров
grid = {
    'C': [],
    'gamma': []
    }

# Будем перебирать все возможные комбинации параметров, которые задали.
param_comb = list(itertools.product(*grid.values()))
for comb in param_comb:
    # Этот словарь можно удобно передавать в объект SVC
    param_dict = dict(zip(grid.keys(), comb))
    # YOUR CODE HERE
    

### Перед отправкой сохраните ноутбук с именем в следующем формате: **02_Фамилия.ipynb**